# Wyszukiwanie semantyczne

Wykorzystuje model `sdadas/mmlw-retrieval-roberta-large` i pgvector

In [1]:
import os

os.chdir("..")

In [2]:
from sentence_transformers import SentenceTransformer

from notes_rag.core.schema import (
    get_engine,
    DbConfig,
    create_session_factory,
    NoteChunkRepository,
)

bi_encoder = SentenceTransformer("sdadas/mmlw-retrieval-roberta-base")

eng = get_engine(DbConfig.local())
sm = create_session_factory(eng)

In [3]:
query = "Jak działa transformata Fouriera?"
query_embedding = bi_encoder.encode(query).tolist()

with sm() as session:
    repo = NoteChunkRepository(session)
    results = repo.search_semantic(query_embedding, limit=5)

for r in results:
    print(r)
    print("-" * 10)
    print(r.content)
    print("\n\n")

NoteChunk(id=848087ae-2c0b-47d3-80d2-e822f9ca2853, title=Szereg i transformata Fouriera, course=wstep-do-multimediow, number=5)
----------
## Przekształcenie Fouriera
* Sygnał okresowy ma widmo dyskretne (prążkowe)
* Prążki na wielokrotnościach częstotliwości podstawowej
* W nieskończoności $T \rightarrow \infty$ następuje uciąglenie widma
	* $T \rightarrow \infty$ - sygnał impulsowy
* Przekształcenie fouriera definiuje transformatę Fouriera (widmo) sygnału $x(t)$
  
$$
X(\omega)
= \mathcal{F}\{x(t)\} 
= \int_{-\infty}^\infty x(t)e^{-j\omega t}dt$$

$$
x(t)
= \mathcal{F}^{-1}\{X(\omega)\}
= \frac{1}{2\pi} \int_{-\infty}^\infty X(\omega)e^{j\omega t} d\omega
$$

## Widmo - pojęcia podstawowe
$$
X(\omega) 
= |X(\omega)|e^{j\arg X(\omega)} 
= \Re(X(\omega)) + j \Im(X(\omega))
$$



NoteChunk(id=043a2820-3585-4235-8e06-d57445d21570, title=Wprowadzenie do teorii sygnałów, course=wstep-do-multimediow, number=10)
----------
* Jest modelem okresowego sygnału próbkującego do próbkowania równomi

In [4]:
from notes_rag.core.retrieval import FulltextRetriever, SemanticRetriever

eng = get_engine(DbConfig.local())
sm = create_session_factory(eng)
session = sm()
repo = NoteChunkRepository(session)
fulltext_retriever = FulltextRetriever(repo)
semantic_retriever = SemanticRetriever.default_model(repo)

In [5]:
query = "Jakie są metody przeciwdziałania przeuczeniu w uczeniu maszynowym"
fulltext_results = fulltext_retriever.retrieve(query, top_k=5)
semantic_results = semantic_retriever.retrieve(query, top_k=5)

print("Wyniki wyszukiwania pełnotekstowego:")
for r in fulltext_results:
    print(r)
    print("-" * 10)
    print(r.content)
    print("\n\n")

print("Wyniki wyszukiwania semantycznego:")
for r in semantic_results:
    print(r)
    print("-" * 10)
    print(r.content)
    print("\n\n")

Wyniki wyszukiwania pełnotekstowego:
Wyniki wyszukiwania semantycznego:
NoteChunk(id=cb3269bd-a777-4c81-a422-fe3f4f0b7e22, title=Klasyfikacja i regresja, course=wstep-do-sztucznej-inteligencji, number=24)
----------
* stosowanie regularyzacji podczas trenowania
	* dropout w modelach neuronowych



NoteChunk(id=7ed49f4e-2860-4742-9011-322050808e69, title=Prywatność i bezpieczeństwo, course=inzynieria-uczenia-maszynowego, number=10)
----------
### Techniki obrony
* Pierwsze, mało skuteczne podejścia
	* dodawanie szumów, dyskretyzacja odpowiedzi - maskowanie gradientów
	* rozszerzyć zbiór uczący o złośliwe dane
	* wykrywanie, czy dany przykład został złośliwie zaburzony
* Złośliwe dane przenoszą się między architekturami i modelami!
	* chyba kwestia podobnych zbiorów treningowych
* Adversarial / robust training
	* inaczej sformułowane zadanie uczenia modelu
	* optymalizuje zachowanie w najgorszym z możliwych przypadków
	* minimalizacja straty przy maksymalnym dozwolonym zaburzeniu (przed 

## Reranking

In [6]:
import torch
from sentence_transformers import CrossEncoder

reranker = CrossEncoder(
    "sdadas/polish-reranker-roberta-v3",
    trust_remote_code=True,
    model_kwargs={"dtype": torch.bfloat16}
)

In [7]:
fulltext_results

[]

In [8]:
semantic_results

In [9]:
query

'Jakie są metody przeciwdziałania przeuczeniu w uczeniu maszynowym'

In [10]:
scores = reranker.predict(
    [(query, r.content) for r in semantic_results]
)
scores

array([0.9921875 , 0.99609375, 0.20800781, 0.9609375 , 0.23828125],
      dtype=float32)

In [11]:
candidates = fulltext_results + semantic_results
ranked = [
    (candidate, score)
    for candidate, score
    in sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
]
ranked

[(<notes_rag.core.schema.NoteChunk at 0x72dd36756c90>, np.float32(0.99609375)),
 (<notes_rag.core.schema.NoteChunk at 0x72dd344ffd10>, np.float32(0.9921875)),
 (<notes_rag.core.schema.NoteChunk at 0x72dd344fc230>, np.float32(0.9609375)),
 (<notes_rag.core.schema.NoteChunk at 0x72dd344fc1a0>, np.float32(0.23828125)),
 (<notes_rag.core.schema.NoteChunk at 0x72dd3676b860>, np.float32(0.20800781))]

In [12]:
from notes_rag.core.retrieval import Reranker

my_reranker = Reranker(reranker)
reranked_results = my_reranker.rerank(query, candidates)
reranked_results